## This is a Demo Notebook of Deploying Air Gapped NIM LLama 3.1-70b with TensorRT profile on AzureML using AzureML CLI

**Prerequisites:**
- AzureML account with minimum 1-A100 GPU VM with 48 vCPUs (NC48ads_A100_v4) provisioned.
- Host machine (CPU only) to download NIM container and model. Needs to have atleast 140 GB  amount of disk space to store the container and model.
- [Azure CLI](https://learn.microsoft.com/en-us/cli/azure/install-azure-cli)
- [Azure ML extension](https://learn.microsoft.com/en-us/azure/machine-learning/how-to-configure-cli?view=azureml-api-2&tabs=public)
- [NGC API Key](https://catalog.ngc.nvidia.com/)

In [ ]:
import subprocess
import requests
import os
import json

In [ ]:
### Fill the configuration variables

subscription_id="XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX"
resource_group="XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX"
workspace="XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX"
location="eastus" # e.g., "southcentralus", "westeurope"
keyvault_name="XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX"
email_address = "youremail@nvidia.com"  
ngc_api_key="XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX"
instance_type="Standard_NC48ads_A100_v4"
acr_registry_name="XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX"
ngc_container = "nvcr.io/nim/meta/llama-3.1-70b-instruct:1.3.3"
endpoint_name = "llama31-70b-nim-trt"
deployment_name = "llama31-70b-nim-trt-tp1"

In [ ]:
def run_command(command, description="", return_output=False):
    try:
        if return_output:
            return subprocess.run(
                command, 
                check=True, 
                capture_output=True,  # capture_output should be set to True directly
                text=True
            ).stdout.strip()            
        else:
            subprocess.run(command, check=True)
            print(f"\033[92mSuccess: {description}\033[0m \n Command: {' '.join(command)}")
    except subprocess.CalledProcessError as e:
        print(f"\033[91mError: {description}\033[0m \n Command: {' '.join(command)}", e)

### Create new workspace

In [ ]:
run_command(["az", "login", "--use-device-code"], "Azure login")

In [ ]:
run_command([
        "az", "ml", "workspace", "create",
        "--name", workspace,
        "--resource-group", resource_group,
        "--location", location
], "AzureML workspace creation")

In [ ]:
run_command([
    "az", "ml", "workspace", "update",
    "--name", workspace,
    "--resource-group", resource_group,
    "--container-registry", f"/subscriptions/{subscription_id}/resourceGroups/{resource_group}/providers/Microsoft.ContainerRegistry/registries/{acr_registry_name}"
    ,"-u",
], "Link Azure Container Registry to Azure ML Workspace")

### Store NGC API KEY in Azure

#### (Option 1) Add NGC API Key to workspace key vault

In [ ]:
workspace_keyvault_uri = run_command(
    ["az", "ml", "workspace", "show", "--name", workspace, "--resource-group", resource_group, "--query", "key_vault", "-o", "tsv"], 
    "Fetching workspace keyvault URI", 
    True
)

keyvault_name = workspace_keyvault_uri.split("/")[-1]

print("Workspace keyvault URI: ", workspace_keyvault_uri)
print("Keyvault name: ", keyvault_name)

# Assign role to allow access to the Key Vault
run_command([
    "az", "role", "assignment", "create",
    "--role", "Key Vault Secrets Officer",
    "--assignee", email_address,
    "--scope", workspace_keyvault_uri
], "Role assignment to access key vault")

# Set a secret in the Key Vault
run_command([
    "az", "keyvault", "secret", "set",
    "--vault-name", keyvault_name,
    "--name", "NGC-KEY",
    "--value", ngc_api_key
], "Add NGC secret to key vault")

# Show the secret in the Key Vault (for verification, if needed)
run_command([
    "az", "keyvault", "secret", "show",
    "--vault-name", keyvault_name,
    "--name", "NGC-KEY"
], "Verify NGC secret in key vault")

#### (Option 2) Add NGC API KEY as an AzureML workspace connection

In [ ]:
# Assign role permission to read secrets from workspace connections
run_command([
    "az", "role", "assignment", "create",
    "--assignee", email_address,
    "--role", "Azure Machine Learning Workspace Connection Secrets Reader",
    "--scope", f"/subscriptions/{subscription_id}/resourcegroups/{resource_group}/providers/Microsoft.MachineLearningServices/workspaces/{workspace}"
], "AzureML secrets reader role assignment")

# Get a personal access token for the workspace
token = run_command(["az", "account", "get-access-token", "--query", "accessToken", "-o", "tsv"], "Getting access token for workspace")

# Define URLs
url = f"https://management.azure.com/subscriptions/{subscription_id}/resourceGroups/{resource_group}/providers/Microsoft.MachineLearningServices/workspaces/{workspace}/connections/ngc?api-version=2023-08-01-preview"
verify_url = f"https://management.azure.com/subscriptions/{subscription_id}/resourceGroups/{resource_group}/providers/Microsoft.MachineLearningServices/workspaces/{workspace}/connections/ngc/listsecrets?api-version=2023-08-01-preview"

# Add a workspace connection to store NGC API key
print("Adding NGC API key to workspace")
headers = {
    "Authorization": f"Bearer {token}",
    "Content-Type": "application/json"
}
data = {
    "properties": {
        "authType": "CustomKeys",
        "category": "CustomKeys",
        "credentials": {
            "keys": {
                "NGC_API_KEY": ngc_api_key
            }
        },
        "expiryTime": None,
        "target": "_",
        "isSharedToAll": False,
        "sharedUserList": []
    }
}

response = requests.put(url, headers=headers, json=data)
if response.status_code == 200:
    print("NGC API key added to workspace successfully.")
else:
    print(f"Failed to add NGC API key to workspace: {response.text}")

# Verify if the key got added
print("Verifying if the NGC API key was added")
verify_response = requests.post(verify_url, headers=headers, json={})
if verify_response.status_code == 200:
    verify_result = verify_response.json()
    ngc_api_key_value = verify_result.get("properties", {}).get("credentials", {}).get("keys", {}).get("NGC_API_KEY")
    
    if ngc_api_key_value == ngc_api_key:
        print("The NGC_API_KEY value matches the provided key.")
    else:
        print("The NGC_API_KEY value does not match the provided key.")
else:
    print(f"Failed to verify NGC API key: {verify_response.text}")

### Pull NIM container

In [ ]:
run_command(["docker", "login", "nvcr.io", "-u", "$oauthtoken", "-p", ngc_api_key], "Docker login")
run_command(["docker", "pull", ngc_container], "Pulling NIM container")

### Cache NIM model for airgapped deployment

#### List all compatible models

In [ ]:
# model_cache_path = "<path-to-cache-model-repo>"
model_cache_path="/home/azureuser/llama-3.1-70b-nim"

# Run the Docker container and list model profiles inside it
docker_run_command = [
    "docker", "run", "--rm", "--name=nim_list_profiles",
    "-e", "LOG_LEVEL=info",
    "-e", "NIM_MANIFEST_ALLOW_UNSAFE",
    "-e", f"NGC_API_KEY={ngc_api_key}",
    "--gpus", "all",
    "-v", f"{model_cache_path}:/model-repo",
    "-u", "root",
    f"{ngc_container}",
    "bash", "-i", "-c", "list-model-profiles"
]

# Execute the command to start Docker and list model profiles
run_command(docker_run_command, "Run Docker container to list model profiles")

# Parse compatible model profile IDs from output (assuming the format provided)

#### Select model profile and download the required NIM model

Select a profile from the compatible profiles from the previous output.

Example:

```
MODEL PROFILES
- Compatible with system and runnable:
  - 395082aa40085d35f004dd3056d7583aea330417ed509b4315099a66cfc72bdd (vllm-bf16-tp2)
  - With LoRA support:
- Compilable to TRT-LLM using just-in-time compilation of HF models to TRTLLM engines:
  - b7b6fa584441d9536091ce5cf80ccc31765780b8a46540da4e7bada5c5108ed9 (tensorrt_llm-trtllm_buildable-bf16-tp2)
  - With LoRA support:
- Incompatible with system:
  - a29dc20fff4ad67746205295ccb4af9e010f8f31207235c75e27786fb834e574 (tensorrt_llm-h100-fp8-tp8-pp1-latency)
  - 852c2c07610526d83d0fa80c656c5ee32c54f91df2626b9e7f7dfb575e25dabf (tensorrt_llm-h100_nvl-fp8-tp4-pp1-latency)
  
...
```
selected_profile = "b7b6fa584441d9536091ce5cf80ccc31765780b8a46540da4e7bada5c5108ed9"

### Select TRT-LLM using just-in-time compilation of HF models to TRTLLM engines: 
profile: b7b6fa584441d9536091ce5cf80ccc31765780b8a46540da4e7bada5c5108ed9 (tensorrt_llm-trtllm_buildable-bf16-tp2)


In [ ]:
# Select a compatible profile ID for the model store creation
# 8b09858c4fc22c360e5e6dda70d67751671c9f5a1182059ebaa91b4babce884c (tensorrt_llm-a100-fp16-tp1-throughput)
selected_profile = "b7b6fa584441d9536091ce5cf80ccc31765780b8a46540da4e7bada5c5108ed9"
print(f"Selected compatible profile ID: {selected_profile}")


```
	mkdir /home/azureuser/llama-3.1-70b-nim
	export LOCAL_NIM_CACHE=/home/azureuser/llama-3.1-70b-nim
	export NIM_MODEL_PROFILE=trt-engine
	
	docker run -it --rm --gpus all --shm-size=16GB -e NIM_CUSTOM_MODEL_NAME=70b-trt-engine -e NGC_API_KEY=xxxxxx  -e NIM_LOW_MEMORY_MODE=1 -v "$LOCAL_NIM_CACHE:/opt/nim/.cache" -u $(id -u) -p 8000:8000  nvcr.io/nim/meta/llama-3.1-70b-instruct:1.3.3

	docker run -it --rm  --gpus all     --shm-size=16GB  -e NIM_MODEL_PROFILE  -e NGC_API_KEY=xxxxxx -v "$LOCAL_NIM_CACHE:/opt/nim/.cache"     -u $(id -u)     -p 8000:8000 nvcr.io/nim/meta/llama-3.1-70b-instruct:1.3.3 bash -i -c list-model-profiles
```

In [ ]:
model_cache_path="/home/azureuser/llama-3.1-70b-nim"
nim_custom_model_name="70b-trt-engine"
# Run the command to create model store for the chosen profile
docker_create_model_command = [
    "docker", "run", "--rm", "--name=nim_model_cache_custom_name",
    "-e", "LOG_LEVEL=info",
    "-e", "NIM_LOW_MEMORY_MODE=1",
    "-e", f"NIM_CUSTOM_MODEL_NAME={nim_custom_model_name}",    
    "-e", f"NGC_API_KEY={ngc_api_key}",
    "--gpus", "all",
    "-v", f"{model_cache_path}:/model-repo",
    "-u", "root",
    f"{ngc_container}"
]

print("Cache lama-3.1-70b-instruct:1.3.3 to model_cache_path command in Docker...")
run_command(docker_create_model_command, "Cache Model command in Docker")

In [ ]:
model_cache_path="/home/azureuser/llama-3.1-70b-nim/local_cache/70b-trt-engine/"

In [ ]:
# Verify if the model was stored successfully
run_command(["ls",model_cache_path], "Model cache verification")

In [ ]:
# Run the Docker container and list model profiles inside it to make sure the new custom model profile is created
docker_run_command = [
    "docker", "run", "--rm", "--name=nim_list_profiles",
    "-e", "LOG_LEVEL=info",
    "-e", "NIM_MODEL_PROFILE=trt-engine",
    "-e", f"NGC_API_KEY={ngc_api_key}",
    "--gpus", "all",
    "-v", f"{model_cache_path}:/model-repo",
    "-u", "root",
    f"{ngc_container}",
    "bash", "-i", "-c", "list-model-profiles"
]

# Execute the command to start Docker and list model profiles
run_command(docker_run_command, "Run Docker container to list model profiles")

### When you run the command above, you should get the newly cached profile listed in list-model-profile command (70b-trt-engine): 
```
942435195ceb8d93a1035c94844200e79e21c1b3744c5b4324ca40b71b164e75 (70b-trt-engine)
```

```
===========================================
	== NVIDIA Inference Microservice LLM NIM ==
	===========================================

	NVIDIA Inference Microservice LLM NIM Version 1.3.3
	Model: meta/llama-3.1-70b-instruct

	......
	Profile 942435195ceb8d93a1035c94844200e79e21c1b3744c5b4324ca40b71b164e75 is not fully defined with checksums
	SYSTEM INFO
	- Free GPUs:
	  -  [20b5:10de] (0) NVIDIA A100 80GB PCIe [current utilization: 0%]
	  -  [20b5:10de] (1) NVIDIA A100 80GB PCIe [current utilization: 0%]
	Profile 0462612f0f2de63b2d423bc3863030835c0fbdbc13b531868670cc416e030029 is not fully defined with checksums
	Profile 09af04392c0375ae5493ca5e6ea0134890ac28f75efd244a57f414f86e97b133 is not fully defined with checksums
	Profile 0b0193d56ec0bba1840ea429993c776f9168a1ca4699e81f4db48319dd7e5c3a is not fully defined with checksums
	Profile 162948dba7374caeb8f7886f7c62a105fd198cfc2dd533aa1cdb34eaea872af0 is not fully defined with checksums
	.......
	MODEL PROFILES
	- Compatible with system and runnable:
	  - 942435195ceb8d93a1035c94844200e79e21c1b3744c5b4324ca40b71b164e75 (70b-trt-engine)
	  - 395082aa40085d35f004dd3056d7583aea330417ed509b4315099a66cfc72bdd (vllm-bf16-tp2)
	  - With LoRA support:
	- Compilable to TRT-LLM using just-in-time compilation of HF models to TRTLLM engines:
	  - b7b6fa584441d9536091ce5cf80ccc31765780b8a46540da4e7bada5c5108ed9 (tensorrt_llm-trtllm_buildable-bf16-tp2)
	  - With LoRA support:
```

#### Push downloaded model to your AzureML workspace. Make sure to point to the local_cache/70b-trt-engine/  path this is where .engine files will be stored
```
	 az ml model create --name llama-3-1-70b-nim-tensorrt-bf16-tp2 --version 1 --path /home/azureuser/llama-3.1-70b-nim/local_cache/70b-trt-engine/ --resource-group rg-azeltov-aml --workspace-name aml_east
```

```
azureuser@nc48a100:~/dev$ ll /home/azureuser/llama-3.1-70b-nim/local_cache/70b-trt-engine/trtllm_engine/
total 140145228
drwxr-xr-x 2 azureuser ubuntu        4096 Jan 15 00:55 ./
drwxr-xr-x 3 azureuser ubuntu        4096 Jan 15 01:03 ../
-rw-r--r-- 1 azureuser ubuntu        3640 Jan 15 00:59 config.json
-rw-r--r-- 1 azureuser ubuntu 71754332004 Jan 15 01:02 rank0.engine
-rw-r--r-- 1 azureuser ubuntu 71754331260 Jan 15 00:59 rank1.engine
```

In [ ]:

cached_model_name = "llama-3-1-70b-nim-tensorrt-bf16-tp2"
model_version = "1"

In [ ]:
run_command([
    "az", "ml", "model", "create",
    "--name", cached_model_name,
    "--version", model_version,
    "--path", model_cache_path,
    "--resource-group", resource_group,
    "--workspace-name", workspace
])

### Push container to Azure Container Registry

In [ ]:
nim_acr_name = ngc_container.replace("nvcr.io", f"{acr_registry_name}.azurecr.io")
print("NIM image name as saved in ACR: ", nim_acr_name)

In [ ]:
# Create Dockerfile content for NIM container
dockerfile_content = f"""FROM {ngc_container}
EXPOSE 8000
USER root
CMD bash -c "echo 'Displaying the NGC API Key:' && echo $NGC_API_KEY && \
             echo 'Displaying the NIM Model Name:' && echo $NIM_MODEL_NAME && \
             echo 'Listing the contents of /model-repo:' && ls -al /model-repo && \
             /opt/nim/start-server.sh"
"""

# Write Dockerfile to disk
with open("Dockerfile", "w") as dockerfile:
    dockerfile.write(dockerfile_content)
print("NIM Dockerfile has been created.")

# Login to Azure Container Registry
print("Logging into Azure Container Registry")
run_command(["az", "acr", "login", "-n", acr_registry_name])

# Build and tag the Docker image
print("Building the new Docker image and tagging it")
run_command(["docker", "build", "-t", nim_acr_name, "-f", "Dockerfile", "."], "Building azure NIM image")

# Clean up Dockerfile after build
#os.remove("Dockerfile")

# Push the image to ACR
print("Pushing the image to ACR")
run_command(["docker", "push", nim_acr_name], "Pushing NIM image to Azure container registry")

### Create AzureML managed endpoint

In [ ]:
# Create the endpoint YAML configuration
endpoint_yaml_content = f"""$schema: https://azuremlschemas.azureedge.net/latest/managedOnlineEndpoint.schema.json
name: {endpoint_name}
auth_mode: key
properties:
    enforce_access_to_default_secret_stores: enabled  # default: disabled
"""

# Write endpoint YAML configuration to file
with open("actual_endpoint_aml.yml", "w") as endpoint_yaml:
    endpoint_yaml.write(endpoint_yaml_content)
print("Endpoint YAML configuration created.")

# Deploy the endpoint using the Azure CLI
print(f"Creating Online Endpoint {endpoint_name}")
output = run_command([
    "az", "ml", "online-endpoint", "create", "-f", "actual_endpoint_aml.yml",
    "--resource-group", resource_group, "--workspace-name", workspace
], return_output=True)

endpointidentityid = ""
if output:
    endpoint_data = json.loads(output)
    endpointidentityid = endpoint_data.get("identity", {}).get("principal_id")
    if endpointidentityid:
        print(f"Principal ID: {endpointidentityid}")
    else:
        print("Principal ID not found.")
else:
    print("No output received.")

# Clean up the generated YAML file
#os.remove("actual_endpoint_aml.yml")
print("Cleaned up temporary endpoint YAML file.")

In [ ]:
# Provide access to endpoint to read secrets from key vault
endpointidentityid = "5ebd626e-907e-463c-ab5a-cdd89702dcfa"
run_command([
    "az", "role", "assignment", "create",
    "--assignee", endpointidentityid,
    "--role", "Key Vault Secrets User",
    "--scope", f"/subscriptions/{subscription_id}/resourcegroups/{resource_group}/providers/Microsoft.KeyVault/vaults/{keyvault_name}"
], "Providing permissions for endpoint to access NGC key from key vault")

### Create NIM deployment

#### Airgapped deployment yaml file

Run this cell to generate a deployment YAML file for deploying NIMs in an airgapped environment. This deployment requires mounting a cached NIM model and setting the NIM container’s NIM_MODEL_NAME environment variable, which instructs NIM to avoid downloading from the internet and instead utilize the model hosted on the instance. The NIM model is sourced by mounting the previously pushed model from our AzureML workspace’s model registry.

In [ ]:
deployment_yaml_content = f"""$schema: https://azuremlschemas.azureedge.net/latest/managedOnlineDeployment.schema.json
name: {deployment_name}
endpoint_name: {endpoint_name}
model: azureml:{cached_model_name}:{model_version}
model_mount_path: /model-repo
environment: 
  name: llama31-70b-nim-trt-tp2-env
  image: {nim_acr_name}
  inference_config:
      liveness_route:
          path: /v1/health/ready
          port: 8000
      readiness_route:
          path: /v1/health/ready
          port: 8000
      scoring_route:
          path: /
          port: 8000
instance_type: {instance_type}
instance_count: 1
request_settings:
    max_concurrent_requests_per_instance: 256
    request_timeout_ms: 180000
environment_variables:
    NIM_MODEL_NAME: /model-repo/70b-trt-engine/
    NGC_API_KEY: {ngc_api_key}
    shm-size: 16GB
    gpus: 0,1
    OMPI_ALLOW_RUN_AS_ROOT: 1
    OMPI_ALLOW_RUN_AS_ROOT_CONFIRM: 1
    # NIM_LOW_MEMORY_MODE: 1
    NIM_MANIFEST_ALLOW_UNSAFE: 1
    NIM_MODEL_PROFILE: trt-engine    
liveness_probe:
    timeout: 300
    period: 300
    failure_threshold: 100
readiness_probe:
    timeout: 300
    period: 300
    failure_threshold: 100    
"""

In [ ]:
# display content of deployment YAML file
import yaml
yaml_content = yaml.safe_load(deployment_yaml_content)
print(yaml.dump(yaml_content, sort_keys=False, default_flow_style=False, indent=2))

In [ ]:
# Write deployment YAML configuration to file
with open("actual_deployment_aml.yml", "w") as deployment_yaml:
    deployment_yaml.write(deployment_yaml_content)
print("Deployment YAML configuration created.")

# Display the modified YAML file for confirmation
with open("actual_deployment_aml.yml", "r") as file:
    print(file.read())

# Create the online deployment using the Azure CLI
print(f"Creating Online Deployment {deployment_name}")
run_command([
    "az", "ml", "online-deployment", "create", "-f", "actual_deployment_aml.yml",
    "--resource-group", resource_group, "--workspace-name", workspace
])

# Clean up the generated YAML file
#os.remove("actual_deployment_aml.yml")
print("Cleaned up temporary deployment YAML file.")

### Test your connection

Verify your deployment using the code below. Modify the code to add your endpoint URL, Endpoint authorization token and AzureML deployment name obtained from "Consume" tab under your AzureML endpoint page as shown below.

![Endpoint details](./endpoint_details.png)

Look at your deployment logs for available serving endpoints and example CURL request

![Serving endpoints](./serving_endpoints.png)

![Example request](./example_request.png)

In [ ]:
base_url = 'https://llama31-70b-nim-trt.eastus.inference.ml.azure.com/' # modify this URL
token = 'xxxxxxxxxxxxxxxx'
model_name = "meta/llama-3.1-70b-instruct"

In [ ]:
import requests
import json
from urllib.parse import urljoin

url = urljoin(base_url, "v1/chat/completions")
headers = {
    'accept': 'application/json',
    'Authorization': f'Bearer {token}', # modify this token
    'Content-Type': 'application/json',
    'azureml-model-deployment': deployment_name
}
data = {
    "messages": [
        {
            "content": "You are a polite and respectful chatbot helping people plan a vacation.",
            "role": "system"
        },
        {
            "content": "What should I do for a 4 day vacation in Spain?",
            "role": "user"
        }
    ],
    "model": model_name,
    "max_tokens": 500,
    "top_p": 1,
    "n": 1,
    "stream": False,
    #"stop": "\n",
    "frequency_penalty": 0.0
}

response = requests.post(url, headers=headers, json=data)
# Pretty print the JSON response
print(json.dumps(response.json(), indent=4))

#### (Optional) Launch a Gradio interface

In [ ]:
# ! pip install gradio

In [ ]:
import gradio as gr
import requests
import json

# Function to handle the chat with LLM
def chat_with_llm(user_message, history):
    # Format messages for the LLM with initial system prompt
    formatted_messages = [
        {"content": "You are a polite and respectful AI assistant.", "role": "system"}
    ]
    
    # Add previous messages from the history
    for user_msg, assistant_msg in history:
        formatted_messages.append({"content": user_msg, "role": "user"})
        formatted_messages.append({"content": assistant_msg, "role": "assistant"})

    # Add the user's latest message
    formatted_messages.append({"content": user_message, "role": "user"})

    data = {
        "messages": formatted_messages,
        "model": model_name,
        "max_tokens": 2000,
        "top_p": 1,
        "n": 1,
        "stream": False,  # Set to True if streaming responses are supported by your API
        "stop": "\n",
        "frequency_penalty": 0.1
    }

    response = requests.post(url, headers=headers, json=data)

    if response.status_code == 200:
        response_data = response.json()
        assistant_reply = response_data.get("choices", [{}])[0].get("message", {}).get("content", "No response")
    else:
        assistant_reply = f"Error: {response.status_code} - {response.text}"

    # Return the assistant's reply as a string
    return assistant_reply

# Create a ChatInterface using Gradio
chat_interface = gr.ChatInterface(
    fn=chat_with_llm,
    title="Multi-Turn LLM Chatbot with gr.ChatInterface",
    description="A chatbot interface that interacts with your LLM endpoint for multi-turn conversations.",
    examples=[["What should I do for a 4-day vacation in Spain?"]]
)

# Launch the interface
chat_interface.launch()